# Improving BARC induction model with RL

## Goal

Create the code to do RL with the BARC induction model. 

Once it works it will be moved to a script.

## Server

Before running the notebook launch a server. 

```bash
export CUDA_VISIBLE_DEVICES=0; trl vllm-serve --max_model_len 12000 --model /home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B
```

## Imports

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # 0 is used by the vllm server

from unsloth import FastLanguageModel
from dataclasses import dataclass
from datasets import Dataset

from trl import GRPOConfig, GRPOTrainer

from arc25.encoders import create_grid_encoder
from arc25.utils import load_arc_dataset_with_solutions
from arc25.data_augmentation import apply_data_augmentation, get_random_data_augmentation_params
from arc25.prompting import create_prompt_from_task
# from arc25.collator import get_data_collator
from arc25.logging import configure_logging, logging
from arc25.parallel_code_execution import run_code_from_predictions

configure_logging()
logger = logging.getLogger(__name__)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-12 16:42:29 [__init__.py:241] Automatically detected platform cuda.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


## First steps

In [2]:
@dataclass
class cfg:
    # base model
    model_path: str = "/home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B"
    load_in_4bit: bool = True
    max_seq_length: int = 12000
    grid_encoder: str = 'ColorNameEncoder()'
    # LoRA
    lora_r: int = 16
    use_rslora: bool = True
    # dataset
    dataset_path: str = "/mnt/hdd0/Kaggle/arc25/data/arc-prize-2024/arc-agi_training_challenges.json"
    output_dir: str = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-12-debug-grpo-b/reward-v1"
    # training hyperparameters
    max_epochs: int = 1
    num_generations: int = 16
    training_batch_size: int = 1
    learning_rate: float = 1e-5

In [3]:
dataset = load_arc_dataset_with_solutions(cfg.dataset_path)
print(f"Loaded {len(dataset)} tasks from {cfg.dataset_path}")

Loaded 400 tasks from /mnt/hdd0/Kaggle/arc25/data/arc-prize-2024/arc-agi_training_challenges.json


In [4]:
llm, tokenizer = FastLanguageModel.from_pretrained(
    cfg.model_path, load_in_4bit=cfg.load_in_4bit,fast_inference=False, max_seq_length=cfg.max_seq_length)
grid_encoder = create_grid_encoder(cfg.grid_encoder)

==((====))==  Unsloth 2025.9.4: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-09-12 16:42:49,454 - arc25.encoders - INFO - create_grid_encoder - Created `ColorNameEncoder()` as grid encoder


Let's create a small dataset.

In [5]:
grpo_dataset = []
for task_id in list(dataset.keys())[:10]: # debug with 10 tasks
    for _ in range(1):
        # params = get_random_data_augmentation_params()
        # task = apply_data_augmentation(dataset[task_id], **params)
        task = dataset[task_id] # debug without data augmentation
        prompt = create_prompt_from_task(
                task, grid_encoder=grid_encoder, tokenizer=tokenizer, shuffle_train_samples=True)
        grpo_dataset.append(dict(prompt=prompt, tasks=task))
grpo_dataset = Dataset.from_list(grpo_dataset)

In [6]:
def arc_reward(completions, tasks, completion_ids, **kwargs):
    """
    Reward function that rewards completions based on how many test cases they pass.

    As input seems to be receiving: completions, prompts, ground_truth and completion_ids
    """
    results = run_code_from_predictions(tasks, list(range(len(completions))), completions, [None]*len(completions), group_results_by_task=False)
    logger.info(f'Completions length: {[len(c) for c in completion_ids]}')
    # logger.info(f"Reward results: {results}")
    # logger.info(f"Task ids: {[result['task_id'] for result in results]}") # this verifies that results are in the same order as completions
    if 'code' in results[0]:
        logger.info(f"Example code:\n{results[0]['code']}")

    rewards = []
    for result in results:
        if 'code' not in result:
            rewards.append(-1.0)
        elif 'train_correct_grids' not in result:
            rewards.append(0.0)
        else:
            rewards.append(float(result['train_correct_grids']) + float(result.get('test_correct_grids', 0)))

    logger.info(f'Rewards: {rewards}')
    return rewards

In [7]:
model = FastLanguageModel.get_peft_model(
    llm,
    r = cfg.lora_r, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = 64,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    use_rslora = cfg.use_rslora,
    # random_state = 3407,
)

Unsloth 2025.9.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
# https://huggingface.co/docs/trl/main/en/grpo_trainer#trl.GRPOConfig
training_args = GRPOConfig(
    output_dir=cfg.output_dir,
    num_train_epochs=cfg.max_epochs,
    per_device_train_batch_size=cfg.training_batch_size,
    num_generations=cfg.num_generations,
    gradient_accumulation_steps=1, # how many prompts are used per task
    learning_rate=cfg.learning_rate,
    # generation
    use_vllm=True,
    vllm_mode="server",
    max_completion_length=1024,
    max_prompt_length=None,
    temperature=1.0,
    top_p=0.95,
    # wandb
    report_to='wandb',
    run_name=os.path.basename(cfg.output_dir),
    # project=os.path.basename(os.path.dirname(cfg.output_dir)),
)
os.environ["WANDB_PROJECT"] = os.path.basename(os.path.dirname(cfg.output_dir))
# set also the output dir for wandb
os.environ["WANDB_DIR"] = cfg.output_dir

print(f"Training arguments: {training_args}")
# Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`. ???
# Why ??? I want to use gradient accumulation to simulate larger batch sizes.

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 16
Training arguments: UnslothGRPOConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
beta=0.001,
bf16=False,
bf16_full_eval=False,
cache_implementation=None,
data_seed=3407,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
delta=None,
disable_dropout=

In [9]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=arc_reward, #reward_num_unique_letters,
    # data_collator=get_data_collator(tokenizer),
    args=training_args,
    train_dataset=grpo_dataset,
    completion_only_loss=True,
)
trainer.train()

2025-09-12 16:42:54,944 - trl.extras.vllm_client - INFO - check_server - Server is up!


INFO 09-12 16:42:55 [__init__.py:1418] Found nccl from library libnccl.so.2
INFO 09-12 16:42:55 [pynccl.py:70] vLLM is using nccl==2.26.2


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: Currently logged in as: guillermobarbadillo to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Executing predictions:   0%|          | 0/1 [00:00<?, ?batch/s]

Executing predictions for batch 0:   0%|          | 0/16 [00:00<?, ?pred/s]

...

2025-09-12 16:48:57,942 - __main__ - INFO - arc_reward - Completions length: [559, 421, 470, 230, 380, 619, 525, 571, 402, 428, 585, 442, 253, 362, 274, 485]
2025-09-12 16:48:57,944 - __main__ - INFO - arc_reward - Example code:
from common import *

import numpy as np
from typing import *

# concepts:
# translation, color swapping, pattern arrangement

# description:
# In the input, you will see a grid with a central square pattern and several colored rectangles surrounding it.
# To create the output, swap the colors of the central square with each surrounding rectangle, then move the rectangles outward in all four cardinal directions until they touch the edges of the grid.

def transform(input_grid: np.ndarray) -> np.ndarray:
    # Step 1: Find the central square and surrounding rectangles
    objects = find_connected_components(input_grid, monochromatic=False)
    
    # Identify the central square (assumed to be the largest connected component)
    central_square = max(objects, key

Executing predictions:   0%|          | 0/1 [00:00<?, ?batch/s]

Executing predictions for batch 0:   0%|          | 0/16 [00:00<?, ?pred/s]

2025-09-12 16:50:17,671 - __main__ - INFO - arc_reward - Completions length: [329, 228, 365, 239, 326, 254, 277, 433, 217, 300, 228, 394, 302, 214, 186, 241]
2025-09-12 16:50:17,673 - __main__ - INFO - arc_reward - Example code:
from common import *

import numpy as np
from typing import *

# concepts:
# scaling, pattern replication, color mapping

# description:
# In the input you will see a 3x3 pattern of colored pixels.
# To create the output, scale the pattern to a 9x9 grid by replicating it three times in each direction,
# and then map the colors of the original pattern to the new scaled positions using a color mapping that 
# shifts the colors by one position in a cyclic manner.

def transform(input_grid):
    # Get the color mapping by extracting the unique colors in the 3x3 pattern
    unique_colors = np.unique(input_grid)
    color_mapping = {color: (unique_colors[(i + 1) % len(unique_colors)]) for i, color in enumerate(unique_colors)}

    # Create the output grid initialized

TrainOutput(global_step=10, training_loss=0.0, metrics={'train_runtime': 461.4252, 'train_samples_per_second': 0.022, 'train_steps_per_second': 0.022, 'total_flos': 0.0, 'train_loss': 0.0})

In [10]:
# use this to reset the vllm server
# ! curl  -X POST --location http://0.0.0.0:8000/close_communicator/

## Debug

There seem to be some compatibility problems:

```
This happens when creating the training conf:
Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`. ???

This happens when training
AttributeError: 'UnslothGRPOConfig' object has no attribute 'delta'

Library versions:
unsloth                   2025.9.1                 pypi_0    pypi
unsloth-zoo               2025.9.2                 pypi_0    pypi
trl                       0.18.0.dev0              pypi_0    pypi

# pip index versions <package-name>
unsloth (2025.9.4)
Available versions: 2025.9.4, 2025.9.3, 2025.9.2, 2025.9.1,
trl (0.23.0)
Available versions: 0.23.0, 0.22.2, 0.22.1, 0.22.0, 0.21.0, 0.20.0, 0.19.1, 0.19.0, 0.18.2, 0.18.1, 0.18.0

I have installed the latest versions of both libraries on the environment `arc25-unsloth`
pip install unsloth==2025.9.4
pip install trl==0.23.0
pip install trl[vllm]

Then it gives this error when launching the server.
NameError: name 'ParallelismConfig' is not defined. Did you mean: 'parallelism_config'?
Solved with: pip install --upgrade accelerate

I also have to remove the collator.
```

This is working, but only did one training step.

```
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 1 | Total steps = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
```

If I reduce the gradient accumulation steps to 1, increase the number of epochs to 3 then it does 30 steps.

Now the problem is that it seems that only be predicting 256 output tokens.

Notice 

So far I hasn't solved any of the training tasks, but seems to be always predicting the code correctly.

## TODO

- [ ] Implement the reward function
- [ ] Check if memory is enough
- [ ] Can I optimize the bouncing in compute between the two gpus?